In [ ]:
import pandas as pd
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import geopandas as gpd
import shapely

# Setup

In [ ]:
brazil_gdf = gpd.read_file('../regions/data/lm_bioma_250.shp')
brazil_gdf = brazil_gdf.to_crs('EPSG:4326')

In [ ]:
if not os.path.isfile('../regions/data/lm_bioma_250_DISSOLVED.shp'):
    brazil_gdf_dissolved = brazil_gdf.copy()
    brazil_gdf_dissolved['geometry'] = brazil_gdf_dissolved.buffer(0.001)
    brazil_gdf_dissolved = brazil_gdf.dissolve()
    brazil_gdf_dissolved.to_file('../regions/data/lm_bioma_250_DISSOLVED.shp')
else:
    brazil_gdf_dissolved = gpd.read_file('../regions/data/lm_bioma_250_DISSOLVED.shp')

In [ ]:
all_csvs = glob.glob('./out/v3_cloudfilt/ls*merged.csv')
all_csvs.sort()

In [ ]:
# Temporary: Convert to lat/lon
for csv in all_csvs:
    out_path = csv.replace('aea', 'wgs84')
    if not os.path.isfile(out_path):
        temp_df = pd.read_csv(csv)
        temp_df = temp_df[['center_lat', 'center_lon', 'hydropoly_max', 'area']]
        gdf = gpd.GeoDataFrame(
            temp_df, geometry=gpd.points_from_xy(temp_df.center_lon, temp_df.center_lat),
            crs='ESRI:102033'
        )
        gdf_wgs84 = gdf.to_crs('EPSG:4326')
        gdf_wgs84['longitude'] = gdf_wgs84.geometry.x
        gdf_wgs84['latitude'] = gdf_wgs84.geometry.y
        gdf_wgs84.drop(columns=['geometry']).to_csv(out_path, index=False)

In [ ]:
def read_process_csv(csv):
    temp_df = pd.read_csv(csv)
    temp_df['satellite'] = os.path.basename(csv)[:3]
    temp_df['year'] = int(os.path.basename(csv)[4:8])
    return temp_df

# Start/end plot

In [ ]:
all_csvs = ['./out/v3_cloudfilt/ls5_1984_cloudfilt_v3_wgs84_merged.csv', './out/v3_cloudfilt/ls7_2019_cloudfilt_v3_wgs84_merged.csv']

In [ ]:
full_df = pd.concat([
    read_process_csv(csv) for csv in all_csvs
])
full_df = full_df.loc[full_df['hydropoly_max']<100]
full_df['area'] = full_df['area']*100/10000
full_df = full_df.loc[full_df['area']<100]

In [ ]:
xlims = brazil_gdf.bounds.min()['minx'], brazil_gdf.bounds.max()['maxx']
xlims = (xlims[0] - 1.5, xlims[1]-4)
ylims = brazil_gdf.bounds.min()['miny'], brazil_gdf.bounds.max()['maxy']
ylims = (ylims[0] - 1, ylims[1] + 1)
ylims_range = ylims[1] - ylims[0]
xlims_range = xlims[1] - xlims[0]

outline_gdf = gpd.GeoDataFrame(
    geometry=gpd.GeoSeries(shapely.geometry.Polygon(
        [[xlims[0], ylims[0]],
         [xlims[0], ylims[1]],
         [xlims[1], ylims[1]],
         [xlims[1], ylims[0]],
         [xlims[0], ylims[0]]])),
    crs='EPSG:4326')
nonbrazil_poly = outline_gdf.overlay(brazil_gdf, how='difference')

In [ ]:
ha_dict = {
    'Amazônia': 'center',
    'Caatinga': 'center',
    'Cerrado': 'center',
    'Mata Atlântica': 'center',
    'Pampa': 'right',
    'Pantanal': 'right'
}
text_colordict = {
    'Amazônia': 'black',
    'Caatinga': 'black',
    'Cerrado': 'black',
    'Mata Atlântica': 'black',
    'Pampa': 'black',
    'Pantanal': 'black'
}
offset_dict = {
    'Amazônia': 0,
    'Caatinga': np.array([2.5,7]),
    'Cerrado': np.array([0, -1]),
    'Mata Atlântica': np.array([4, -6.2]),
    'Pampa': np.array([-2, -2.2]),
    'Pantanal': np.array([-2, -1])
}


In [ ]:
axes_height_ratios=[1, 0.04]
fig, axs = plt.subplots(2, 2, figsize = (7.5, 4.5),
                       gridspec_kw={"height_ratios":axes_height_ratios})

# Plot 1984 first
year_1984_df = full_df.loc[full_df['year']==1984]
axs[0,0].hexbin(year_1984_df['longitude'], year_1984_df['latitude'],
             gridsize=32,
             vmin=0, vmax=10000,
             extent=xlims + ylims,
             cmap='GnBu',
             linewidths=0.2,
             edgecolor='none')


# Plot 2022
year_2022_df = full_df.loc[full_df['year']==2019]
im = axs[0, 1].hexbin(year_2022_df['longitude'], year_2022_df['latitude'],
              gridsize=32,
             vmin=0, vmax=10000,
             extent=xlims + ylims,
             cmap='GnBu',
             linewidths=0.1,
             edgecolor='none')


for cur_ax in axs[0]:
    cur_ax.set_xlabel('Lon (deg)')
    cur_ax.set_ylabel('Lat (deg)')
    cur_ax.set_xlim(xlims)
    cur_ax.set_ylim(ylims)
axs[0,0].set_title('1984')
axs[0,1].set_title('2019')

# Set up boundaries and labels
for ax in axs[0]:
    brazil_gdf.boundary.plot(ax=ax, color='black', alpha=0.3, lw=0.3)
    brazil_gdf_dissolved.boundary.plot(ax=ax, color='black', alpha=0.7, lw=0.5)
brazil_gdf.apply(lambda x: axs[0, 0].annotate(text=x['Bioma'].replace(' ','\n'), 
                                                xy=np.array(x.geometry.centroid.coords[0]) + offset_dict[x['Bioma']],
                                                ha=ha_dict[x['Bioma']],
                                                color=text_colordict[x['Bioma']]), axis=1)

# Remove outside of Brazil
nonbrazil_poly.plot(ax=axs[0,1],color='white')
nonbrazil_poly.plot(ax=axs[0,0],color='white')

# Set up colorbar
gs = axs[0, 0].get_gridspec()
for ax in axs[-1]:
    ax.remove()
axbig = fig.add_subplot(gs[-1, :])
axbig.set_title('Reservoir Count (per km$^2$)')
for i, label in enumerate(['$(a)$', '$(b)$']):
    axs[0, i].annotate(
            label,
            xy=(0, 1), xycoords='axes fraction',
            xytext=(0.3, -1.5), textcoords='offset fontsize',
            fontsize=12, verticalalignment='bottom', fontfamily='serif')

# Make colorbar per-km2
print('Max value:',im.get_array().data.max())
vmax = im.get_clim()[1]
tick_increment = vmax/3
hex_area = (outline_gdf.to_crs('ESRI:102033').area.values[0]/len(im.get_array()))/(1000*1000)
print('Hex area:', hex_area)
print('Max cbar density:', vmax/hex_area)
cb = fig.colorbar(im, cax=axbig, orientation='horizontal',
                  ticks=[0,tick_increment, 2*tick_increment,vmax])
# Adjust based on max density                  
cb.ax.set_xticklabels([0, 0.2, 0.4, 0.6])
fig.tight_layout()
plt.savefig('/home/ksolvik/research/reservoirs/figs/ch1/all_brazil_map.jpg', dpi=300)

# Animation

In [ ]:
year_dict = {
   'ls5': np.arange(1984, 2002),
   'ls7': np.arange(2002, 2020),
   'ls8': np.arange(2020, 2024) 
}

In [ ]:
all_csvs = []
for ls in year_dict.keys():
    for y in year_dict[ls]:
        all_csvs.append('./out/v3_cloudfilt/{}_{}_cloudfilt_v3_wgs84_merged.csv'.format(ls, y))

In [ ]:
full_df = pd.concat([
    read_process_csv(csv) for csv in all_csvs
])
full_df = full_df.loc[full_df['hydropoly_max']<100]
full_df['area'] = full_df['area']*100/10000
full_df = full_df.loc[full_df['area']<100]

In [ ]:
%matplotlib notebook

fig, ax = plt.subplots(figsize=(12,12))
ax.set_xlim(xlims)
ax.set_ylim(ylims)
ax.set_xlabel('Lon (deg)')
ax.set_ylabel('Lat (deg)')

# Remove outside of Brazil
year_start = 1984
year_end = 2019
frame_list = np.concatenate([[year_start]*5, np.arange(year_start, year_end+1), [year_end]*5])
brazil_gdf.boundary.plot(ax=ax, color='black', alpha=0.3, lw=0.3)
brazil_gdf_dissolved.boundary.plot(ax=ax, color='black', alpha=0.7, lw=0.5)

def animate(y):
    year_df = full_df.loc[full_df['year']==y]
    if y != year_start:
        ax.collections[-1].remove()
    hexlayer=ax.hexbin(year_df['longitude'], year_df['latitude'],
                # gridsize=32,
                # vmin=0, vmax=10000,
                gridsize=80,
                vmin=0, vmax=2000,
                extent=xlims + ylims,
                cmap='GnBu',
                linewidths=0.1,
                edgecolor='none')
    nonbrazil_poly.plot(ax=ax,color='white')
    ax.set_title(y)
    return hexlayer,

ani = FuncAnimation(fig, animate, frames=frame_list, interval=500)
ani.save('../../../../../figs/ch2/frontier_really_small.gif')